In [ ]:
import pandas as pd
import json
from tqdm import tqdm
from os import walk

from scipy import stats
import numpy as np

from scipy import stats

In [ ]:
data_path = "./LLM_scored/run0config/"
filenames = list(set(next(walk(data_path), (None,None,[]))[2]))

495


In [ ]:
layer_list = ['layer_original', 'layer_1', 'layer_23', 'layer_-1', 'layer_last']
tradi_list = ['authenticity', 'tradition', 'prototype']
novel_list = ['novelty', 'uniquness', 'newness', 'difference', 'surprise', 'creativity', 'originality']

# Score difference between traiditionnal and Creative Keywords

In [4]:

tradi_new, tradi_uniq, tradi_diff, tradi_nsurp, tradi_dsurp = [], [], [], [], []
novel_new, novel_uniq, novel_diff, novel_nsurp, novel_dsurp = [], [], [], [], []
for k in tqdm(range(len(filenames))):
    recette = filenames[k]
    file_path = data_path + recette
    with open(file_path) as json_file:
        recipe_dict = json.load(json_file)

    if 'same_country_novelty' in recipe_dict['LLM_gen']:
        llm_recipe_ids = list(recipe_dict['LLM_gen']['same_country_novelty'].keys())
        for i in range(len(llm_recipe_ids)):
            tradi_novel_text = llm_recipe_ids[i].split('_')[-1]

            if tradi_novel_text in tradi_list: 
                    recipe = recipe_dict['LLM_gen']['same_country_novelty'][llm_recipe_ids[i]]
                    tradi_new.append(recipe['layer_original']['newness_div'])
                    tradi_uniq.append(recipe['layer_original']['uniq_dist'])
                    tradi_diff.append(recipe['layer_original']['diff_local'])
                    tradi_nsurp.append(recipe['layer_original']['new_surprise'])
                    tradi_dsurp.append(recipe['layer_original']['dist_surprise'])

            if tradi_novel_text in novel_list: 
                recipe = recipe_dict['LLM_gen']['same_country_novelty'][llm_recipe_ids[i]]
                novel_new.append(recipe['layer_original']['newness_div'])
                novel_uniq.append(recipe['layer_original']['uniq_dist'])
                novel_diff.append(recipe['layer_original']['diff_local'])
                novel_nsurp.append(recipe['layer_original']['new_surprise'])
                novel_dsurp.append(recipe['layer_original']['dist_surprise'])

  0%|          | 0/495 [00:00<?, ?it/s]

100%|██████████| 495/495 [00:37<00:00, 13.28it/s]


In [ ]:
#EStimating t-test
# tradi_new | tradi_uniq | tradi_diff | tradi_nsurp | tradi_dsurp
# novel_new | novel_uniq | novel_diff | novel_nsurp | novel_dsurp
x = np.array(tradi_new)
y = np.array(novel_new)

# Step 1: Mean values
mean_x, mean_y = np.mean(x), np.mean(y)
print("Mean TRADI:", mean_x, "Mean NOVEL:", mean_y)

# Step 2a: Independent t-test (Welch’s by default with equal_var=False)
t_stat, p_val_t = stats.ttest_ind(x, y, equal_var=False)
print("t-test p-value:", p_val_t)

# Step 2b: Mann–Whitney U test
u_stat, p_val_u = stats.mannwhitneyu(x, y, alternative='two-sided')
print("Mann–Whitney p-value:", p_val_u)

Mean TRADI: 0.0060757633859687896 Mean NOVEL: 0.007264306983751179
t-test p-value: 1.4388144737905194e-15
Mann–Whitney p-value: 5.501405205549079e-16


# Measuring differences between Same Country vs Variation country

In [10]:
same_new, same_uniq, same_diff, same_nsurp, same_dsurp = [], [], [], [], []
varia_new, varia_uniq, varia_diff, varia_nsurp, varia_dsurp = [], [], [], [], []
for k in tqdm(range(len(filenames))):
    recette = filenames[k]
    file_path = data_path + recette
    with open(file_path) as json_file:
        recipe_dict = json.load(json_file)

    if 'same_country_novelty' in recipe_dict['LLM_gen']:
        llm_recipe_ids = list(recipe_dict['LLM_gen']['same_country_novelty'].keys())
        for i in range(len(llm_recipe_ids)):
            tradi_novel_text = llm_recipe_ids[i].split('_')[-1]

            if tradi_novel_text in novel_list: 
                recipe = recipe_dict['LLM_gen']['same_country_novelty'][llm_recipe_ids[i]]
                same_new.append(recipe['layer_original']['newness_div'])
                same_uniq.append(recipe['layer_original']['uniq_dist'])
                same_diff.append(recipe['layer_original']['diff_local'])
                same_nsurp.append(recipe['layer_original']['new_surprise'])
                same_dsurp.append(recipe['layer_original']['dist_surprise'])

        
    varia_countries = list(recipe_dict['LLM_gen']['variation_novelty'].keys())
    for varia in varia_countries:
        llm_recipe_varia_ids = list(recipe_dict['LLM_gen']['variation_novelty'][varia].keys())
        for i in range(len(llm_recipe_varia_ids)):
            varia_novel_text = llm_recipe_varia_ids[i].split('_')[-1]
            if varia_novel_text in novel_list: 
                recipe = recipe_dict['LLM_gen']['variation_novelty'][varia][llm_recipe_varia_ids[i]]
                varia_new.append(recipe['layer_original']['newness_div'])
                varia_uniq.append(recipe['layer_original']['uniq_dist'])
                varia_diff.append(recipe['layer_original']['diff_local'])
                varia_nsurp.append(recipe['layer_original']['new_surprise'])
                varia_dsurp.append(recipe['layer_original']['dist_surprise'])

100%|██████████| 495/495 [00:40<00:00, 12.20it/s]


In [ ]:
#T-TEST BY GROUPS
# same_new | same_uniq | same_diff | same_nsurp | same_dsurp
# varia_new | varia_uniq | varia_diff | varia_nsurp | varia_dsurp
x = np.array(same_new)
y = np.array(varia_new)

# Step 1: Mean values
mean_x, mean_y = np.mean(x), np.mean(y)
print("Mean SAME:", mean_x, "Mean VARIA:", mean_y)

# Step 2a: Independent t-test (Welch’s by default with equal_var=False)
t_stat, p_val_t = stats.ttest_ind(x, y, equal_var=False)
print("t-test p-value:", p_val_t)

# Step 2b: Mann–Whitney U test
u_stat, p_val_u = stats.mannwhitneyu(x, y, alternative='two-sided')
print("Mann–Whitney p-value:", p_val_u)

Mean SAME: 0.007264306983751179 Mean VARIA: 0.007604035003525564
t-test p-value: 0.0002168907781678741
Mann–Whitney p-value: 2.454363385543521e-78


# Average for each keywords and correlation with cultural distances

In [16]:
novel_new, novel_uniq, novel_diff, novel_nsurp, novel_dsurp = [], [], [], [], []
uniq_new, uniq_uniq, uniq_diff, uniq_nsurp, uniq_dsurp = [], [], [], [], []
new_new, new_uniq, new_diff, new_nsurp, new_dsurp = [], [], [], [], []
diff_new, diff_uniq, diff_diff, diff_nsurp, diff_dsurp = [], [], [], [], []
surp_new, surp_uniq, surp_diff, surp_nsurp, surp_dsurp = [], [], [], [], []
crea_new, crea_uniq, crea_diff, crea_nsurp, crea_dsurp = [], [], [], [], []
orig_new, orig_uniq, orig_diff, orig_nsurp, orig_dsurp = [], [], [], [], []

test = []

for k in tqdm(range(len(filenames))):
    recette = filenames[k]
    file_path = data_path + recette
    with open(file_path) as json_file:
        recipe_dict = json.load(json_file)
    #defined_ = False
    if 'same_country_novelty' in recipe_dict['LLM_gen']:
        llm_recipe_ids = list(recipe_dict['LLM_gen']['same_country_novelty'].keys())
        for i in range(len(llm_recipe_ids)):
            defined_ = False
            tradi_novel_text = llm_recipe_ids[i].split('_')[-1]
            if 'defined' in llm_recipe_ids[i].split('_'):
                defined_ = True

            #print(llm_recipe_ids[i].split('_'))  #defined
            if tradi_novel_text == 'novelty': 
                recipe = recipe_dict['LLM_gen']['same_country_novelty'][llm_recipe_ids[i]]
                novel_new.append(recipe['layer_original']['newness_div'])
                novel_uniq.append(recipe['layer_original']['uniq_dist'])
                novel_diff.append(recipe['layer_original']['diff_local'])
                novel_nsurp.append(recipe['layer_original']['new_surprise'])
                novel_dsurp.append(recipe['layer_original']['dist_surprise'])
            if tradi_novel_text == 'uniquness': 
            #if tradi_novel_text == 'uniquness' and defined_: 
                recipe = recipe_dict['LLM_gen']['same_country_novelty'][llm_recipe_ids[i]]
                uniq_new.append(recipe['layer_original']['newness_div'])
                uniq_uniq.append(recipe['layer_original']['uniq_dist'])
                uniq_diff.append(recipe['layer_original']['diff_local'])
                uniq_nsurp.append(recipe['layer_original']['new_surprise'])
                uniq_dsurp.append(recipe['layer_original']['dist_surprise'])

            if tradi_novel_text  == 'newness': 
            #if tradi_novel_text == 'newness' and defined_: 
                recipe = recipe_dict['LLM_gen']['same_country_novelty'][llm_recipe_ids[i]]
                new_new.append(recipe['layer_original']['newness_div'])
                new_uniq.append(recipe['layer_original']['uniq_dist'])
                new_diff.append(recipe['layer_original']['diff_local'])
                new_nsurp.append(recipe['layer_original']['new_surprise'])
                new_dsurp.append(recipe['layer_original']['dist_surprise'])
            if tradi_novel_text == 'difference': 
            #if tradi_novel_text == 'difference' and defined_: 
                recipe = recipe_dict['LLM_gen']['same_country_novelty'][llm_recipe_ids[i]]
                diff_new.append(recipe['layer_original']['newness_div'])
                diff_uniq.append(recipe['layer_original']['uniq_dist'])
                diff_diff.append(recipe['layer_original']['diff_local'])
                diff_nsurp.append(recipe['layer_original']['new_surprise'])
                diff_dsurp.append(recipe['layer_original']['dist_surprise'])
            if tradi_novel_text == 'surprise': 
            #if tradi_novel_text == 'surprise' and defined_: 
                recipe = recipe_dict['LLM_gen']['same_country_novelty'][llm_recipe_ids[i]]
                surp_new.append(recipe['layer_original']['newness_div'])
                surp_uniq.append(recipe['layer_original']['uniq_dist'])
                surp_diff.append(recipe['layer_original']['diff_local'])
                surp_nsurp.append(recipe['layer_original']['new_surprise'])
                surp_dsurp.append(recipe['layer_original']['dist_surprise'])
            if tradi_novel_text == 'creativity': 
                recipe = recipe_dict['LLM_gen']['same_country_novelty'][llm_recipe_ids[i]]
                crea_new.append(recipe['layer_original']['newness_div'])
                crea_uniq.append(recipe['layer_original']['uniq_dist'])
                crea_diff.append(recipe['layer_original']['diff_local'])
                crea_nsurp.append(recipe['layer_original']['new_surprise'])
                crea_dsurp.append(recipe['layer_original']['dist_surprise'])
            if tradi_novel_text == 'originality': 
                recipe = recipe_dict['LLM_gen']['same_country_novelty'][llm_recipe_ids[i]]
                orig_new.append(recipe['layer_original']['newness_div'])
                orig_uniq.append(recipe['layer_original']['uniq_dist'])
                orig_diff.append(recipe['layer_original']['diff_local'])
                orig_nsurp.append(recipe['layer_original']['new_surprise'])
                orig_dsurp.append(recipe['layer_original']['dist_surprise'])
        
    varia_countries = list(recipe_dict['LLM_gen']['variation_novelty'].keys())
    for varia in varia_countries:
        llm_recipe_varia_ids = list(recipe_dict['LLM_gen']['variation_novelty'][varia].keys())
        for i in range(len(llm_recipe_varia_ids)):
            defined_ = False
            varia_novel_text = llm_recipe_varia_ids[i].split('_')[-1]
           # print(varia_novel_text)
            if 'defined' in llm_recipe_varia_ids[i].split('_'):
                defined_ = True 

            if varia_novel_text == 'novelty': 
                recipe = recipe_dict['LLM_gen']['variation_novelty'][varia][llm_recipe_varia_ids[i]]
                novel_new.append(recipe['layer_original']['newness_div'])
                novel_uniq.append(recipe['layer_original']['uniq_dist'])
                novel_diff.append(recipe['layer_original']['diff_local'])
                novel_nsurp.append(recipe['layer_original']['new_surprise'])
                novel_dsurp.append(recipe['layer_original']['dist_surprise'])
            if varia_novel_text in ['uniquness', 'unique']: 
            #if varia_novel_text in ['uniquness', 'unique'] and defined_: 
                recipe = recipe_dict['LLM_gen']['variation_novelty'][varia][llm_recipe_varia_ids[i]]
                uniq_new.append(recipe['layer_original']['newness_div'])
                uniq_uniq.append(recipe['layer_original']['uniq_dist'])
                uniq_diff.append(recipe['layer_original']['diff_local'])
                uniq_nsurp.append(recipe['layer_original']['new_surprise'])
                uniq_dsurp.append(recipe['layer_original']['dist_surprise'])

            if varia_novel_text in ['new','newness'] : 
            #if varia_novel_text in ['new','newness'] and defined_: 
                recipe = recipe_dict['LLM_gen']['variation_novelty'][varia][llm_recipe_varia_ids[i]]
                new_new.append(recipe['layer_original']['newness_div'])
                new_uniq.append(recipe['layer_original']['uniq_dist'])
                new_diff.append(recipe['layer_original']['diff_local'])
                new_nsurp.append(recipe['layer_original']['new_surprise'])
                new_dsurp.append(recipe['layer_original']['dist_surprise'])
            
            if varia_novel_text in ['difference', 'different']:
            #if varia_novel_text in ['difference', 'different'] and defined_: 
                recipe = recipe_dict['LLM_gen']['variation_novelty'][varia][llm_recipe_varia_ids[i]]
                diff_new.append(recipe['layer_original']['newness_div'])
                diff_uniq.append(recipe['layer_original']['uniq_dist'])
                diff_diff.append(recipe['layer_original']['diff_local'])
                diff_nsurp.append(recipe['layer_original']['new_surprise'])
                diff_dsurp.append(recipe['layer_original']['dist_surprise'])
            #
            if varia_novel_text in ['surprise','surprising']:
            #if varia_novel_text in ['surprise','surprising'] and defined_: 
                recipe = recipe_dict['LLM_gen']['variation_novelty'][varia][llm_recipe_varia_ids[i]]
                surp_new.append(recipe['layer_original']['newness_div'])
                surp_uniq.append(recipe['layer_original']['uniq_dist'])
                surp_diff.append(recipe['layer_original']['diff_local'])
                surp_nsurp.append(recipe['layer_original']['new_surprise'])
                surp_dsurp.append(recipe['layer_original']['dist_surprise'])
            if varia_novel_text == 'creative, desirable and useful':
                recipe = recipe_dict['LLM_gen']['variation_novelty'][varia][llm_recipe_varia_ids[i]]
                crea_new.append(recipe['layer_original']['newness_div'])
                crea_uniq.append(recipe['layer_original']['uniq_dist'])
                crea_diff.append(recipe['layer_original']['diff_local'])
                crea_nsurp.append(recipe['layer_original']['new_surprise'])
                crea_dsurp.append(recipe['layer_original']['dist_surprise'])
            
            if varia_novel_text == 'originality': 
                recipe = recipe_dict['LLM_gen']['variation_novelty'][varia][llm_recipe_varia_ids[i]]
                orig_new.append(recipe['layer_original']['newness_div'])
                orig_uniq.append(recipe['layer_original']['uniq_dist'])
                orig_diff.append(recipe['layer_original']['diff_local'])
                orig_nsurp.append(recipe['layer_original']['new_surprise'])
                orig_dsurp.append(recipe['layer_original']['dist_surprise'])
            

  0%|          | 0/495 [00:00<?, ?it/s]

100%|██████████| 495/495 [00:40<00:00, 12.24it/s]


In [23]:
x_new = np.array(diff_new)  # novel_new | uniq_new | new_new | diff_new | surp_new | crea_new | orig_new
x_uniq = np.array(diff_uniq)  # novel_uniq | uniq_uniq | new_uniq | diff_uniq | surp_uniq | crea_uniq | orig_uniq
x_diff = np.array(diff_diff)  # novel_diff | uniq_diff | new_diff | diff_diff | surp_diff | crea_diff | orig_diff
x_nsurp = np.array(diff_nsurp) # novel_nsurp | uniq_nsurp | new_nsurp | diff_nsurp | surp_nsurp | crea_nsurp | orig_nsurp
x_dsurp = np.array(diff_dsurp) # novel_dsurp | uniq_dsurp | new_dsurp | diff_dsurp | surp_dsurp | crea_dsurp | orig_dsurp
#orig_uniq crea_dsurp  surp_diff   new_nsurp  uniq_new   novel_ diff_

# Step 1: Mean values
mean_x_new = np.mean(x_new)
meanx_uniq = np.mean(x_uniq)
mean_x_diff = np.mean(x_diff)
mean_x_nsurp = np.mean(x_nsurp)
mean_x_dsurp = np.mean(x_dsurp)

print("Mean NEWNESS:", mean_x_new)
print("Mean UNIQUENESS:", meanx_uniq)
print("Mean DIFFERENCE:", mean_x_diff)
print("Mean N SURPRISE:", mean_x_nsurp)
print("Mean D SURPRISE:", mean_x_dsurp)

Mean NEWNESS: 0.007213754844379219
Mean UNIQUENESS: 0.5412956958920554
Mean DIFFERENCE: 0.997150232009907
Mean N SURPRISE: 0.969912812284875
Mean D SURPRISE: 0.5510231396386027
